In [ ]:
from dotenv import load_dotenv
import os

from langchain_openai import OpenAIEmbeddings

GEOCODING_API_KEY = os.getenv("GEOCODING_API_KEY")
print(GEOCODING_API_KEY)

In [ ]:
import pandas as pd
import requests
import time

df = pd.read_csv("data.csv")

batch_size = 1000
start = 8231 

success = 0
fail = 0
for i in range(start, start + batch_size):
  row = df.iloc[i]
  address = row["name"] + " " + (row["location"] or row["city"])
  url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={GEOCODING_API_KEY}"
  res = requests.get(url)
  data = res.json()

  if data["status"] == "OK" and data["results"] is not None:
    lat = data["results"][0]["geometry"]["location"]["lat"]
    lng = data["results"][0]["geometry"]["location"]["lng"]
    # print(lat, lng)
    success += 1
  else:
    fail += 1
    print(data) # print error message

    lat = None
    lng = None
  
  print(f"{i}: {success} success, {fail} fail")
  df.at[i, "lat"] = lat
  df.at[i, "lng"] = lng

  # sleep
  time.sleep(0.05)

df.to_csv("data.csv", index=False)

In [ ]:
df.to_csv("data.csv", index=False)